In [46]:
import tensorflow as tf
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from scipy import ndimage
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import vgg

%matplotlib inline

In [1]:
import dataset_utils
root_dataset_directory = './dataset/'

# Convert image dataset to TFRecord format
dataset_utils.process_directory(root_dataset_directory)

./dataset/train
Dataset label map: {'fiat_125p': 0, 'fiat_126p': 1, 'polonez_caro': 2}
Converting train dataset from: ./dataset/train
Have 589 per record file
Writing ./dataset/train-0.tfrecords
Processing sample 589 of 589Converting validation dataset from: ./dataset/validation
Have 72 per record file
Writing ./dataset/validation-0.tfrecords
Processing sample 72 of 72Converting test dataset from: ./dataset/test
Have 76 per record file
Writing ./dataset/test-0.tfrecords
Processing sample 76 of 76

In [24]:
def _deserialize_image_record(record):
    feature_map = {
            'label': tf.FixedLenFeature([ ], tf.int64,-1),
            'text_label': tf.FixedLenFeature([ ], tf.string, ''),
            'image': tf.FixedLenFeature([ ], tf.string, ''),
    }
    with tf.name_scope('deserialize_image_record'):
        obj = tf.parse_single_example(record, feature_map)
        imgdata = obj['image']
        label   = tf.cast(obj['label'], tf.int32)
        text_label =  obj['text_label']
        return imgdata, label, text_label

SIZE = 224
NUM_CHANNELS = 3

def preprocess_image_record(record, size, num_channels, is_training=False):

    print(record)
    imgdata, label, text = _deserialize_image_record(record)
    print(imgdata, label, text)
    image = tf.image.decode_jpeg(imgdata, channels=NUM_CHANNELS,
                                fancy_upscaling=False,
                                dct_method='INTEGER_FAST')

    image = tf.image.resize_images(image, [size, size], 
                               method=tf.image.ResizeMethod.BILINEAR,
                                align_corners=False)
    if is_training:
        # For training, we want to randomize some of the distortions.
        image = tf.image.random_flip_left_right(image)


    #image = tf.image.resize_bilinear(image, size)
    image = tf.image.per_image_standardization(image)
    return image, label


def load_tfrecord_dataset(filenames, is_training):
    shuffle_buffer_size = 1000
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.shuffle(shuffle_buffer_size)
    preproc_func = lambda record, : preprocess_image_record(
        record, SIZE, NUM_CHANNELS, is_training)

    dataset = dataset.map(map_func=preproc_func)
    dataset = dataset.repeat()  
    dataset = dataset.batch(8)
    
    return dataset

In [ ]:
image_size = 224
num_classes = 3
is_training = True

pre_trained_model_path = '/workspace/pre_trained_models/vgg_16.ckpt'
tf.logging.set_verbosity(tf.logging.INFO)

def model_fn(images, labels, num_classes, mode):
    with tf.contrib.slim.arg_scope(vgg.vgg_arg_scope()):
        logits, end_points = vgg.vgg_16(images, num_classes, is_training=is_training)
        predictions = {
          'classes': tf.argmax(input=logits, axis=1),
          'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
        }
        
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
        
        accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
        tf.summary.scalar('accuracy', accuracy[1])
        
        exclude = ['vgg_16/fc8']
        variables_to_restore = tf.contrib.slim.get_variables_to_restore(exclude=exclude)
        #print(variables_to_restore)
        scopes = { os.path.dirname(v.name) for v in variables_to_restore }
        tf.train.init_from_checkpoint(pre_trained_model_path, 
                              {v.name.split(':')[0]: v for v in variables_to_restore})
        
         
        fc8_variables = tf.contrib.framework.get_variables('vgg_16/fc8')
        fc8_init = tf.variables_initializer(fc8_variables)

         # We can then call the total loss easily
        tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        loss = tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=labels)  
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        total_loss = tf.add_n([loss] + reg_losses, name='total_loss')    #obtain the regularization losses as well
        
        if mode == tf.estimator.ModeKeys.EVAL:
            metrics = {'eval_accuracy': accuracy}
            return tf.estimator.EstimatorSpec(
                mode, loss=loss, eval_metric_ops=metrics)

        assert(mode == tf.estimator.ModeKeys.TRAIN)

        global_step = tf.train.get_or_create_global_step()
        optimizer = tf.train.AdamOptimizer(learning_rate=0.00016)
        train_op = optimizer.minimize(total_loss, global_step, var_list=fc8_variables)
        logging_hook = tf.train.LoggingTensorHook({"loss" : loss, 'total_loss' : total_loss}, every_n_iter=10)
     
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            loss=total_loss,
            train_op=train_op,
            training_hooks = [logging_hook])

# Create the Estimator
classifier = tf.estimator.Estimator(
    model_fn=lambda features, labels, mode: model_fn(features, labels, num_classes, mode),
    model_dir='/workspace/trained_model',
    config=tf.estimator.RunConfig(
        save_summary_steps=100,
        save_checkpoints_steps=500
    )
)

batch_size = 8
train_examples = 589
steps_per_epoch = train_examples/batch_size
num_epochs = 200

# Train the model  
train_filenames = ['/workspace/dataset/train-0.tfrecords']
train_input_fn = lambda: load_tfrecord_dataset(train_filenames, is_training=True)

eval_filenames =  ['/workspace/dataset/validation-0.tfrecords']
eval_input_fn = lambda: load_tfrecord_dataset(eval_filenames, is_training=False)



for i in range(num_epochs):
    print("Training epoch: {}".format(i+1))
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=steps_per_epoch*(i+1))
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)
    tf.estimator.train_and_evaluate(classifier,train_spec,eval_spec)


INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8a607ce518>, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/workspace/trained_model', '_num_worker_replicas': 1, '_session_config': None, '_save_checkpoints_steps': 500, '_num_ps_replicas': 0, '_train_distribute': None, '_tf_random_seed': None, '_evaluation_master': '', '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_is_chief': True, '_master': '', '_global_id_in_cluster': 0, '_save_checkpoints_secs': None, '_task_id': 0, '_device_fn': None, '_task_type': 'worker', '_service': None, '_save_summary_steps': 100}
Training epoch: 1
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 500 or save_checkpoints_secs None.
Tensor("arg0:0", shape=(), dtype=strin

In [53]:
validate_filenames =  ['/workspace/dataset/validate-0.tfrecords']
classifier.evaluate(input_fn=lambda: load_tfrecord_dataset(train_filenames, is_training=False),steps=100)

Tensor("arg0:0", shape=(), dtype=string, device=/device:CPU:0)
Tensor("deserialize_image_record/ParseSingleExample/ParseSingleExample:0", shape=(), dtype=string, device=/device:CPU:0) Tensor("deserialize_image_record/Cast:0", shape=(), dtype=int32, device=/device:CPU:0) Tensor("deserialize_image_record/ParseSingleExample/ParseSingleExample:2", shape=(), dtype=string, device=/device:CPU:0)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-13-22:16:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /workspace/trained_model/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evalua

{'global_step': 500, 'loss': 2.175391}

In [54]:
test_filenames = ['/workspace/dataset/test-0.tfrecords']
for prediction in classifier.predict(input_fn=lambda: load_tfrecord_dataset(test_filenames, is_training=False)):
    print(prediction)

Tensor("arg0:0", shape=(), dtype=string, device=/device:CPU:0)
Tensor("deserialize_image_record/ParseSingleExample/ParseSingleExample:0", shape=(), dtype=string, device=/device:CPU:0) Tensor("deserialize_image_record/Cast:0", shape=(), dtype=int32, device=/device:CPU:0) Tensor("deserialize_image_record/ParseSingleExample/ParseSingleExample:2", shape=(), dtype=string, device=/device:CPU:0)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /workspace/trained_model/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'probabilities': array([0.45348924, 0.34054807, 0.20596269], dtype=float32), 'classes': 0}
{'probabilities': array([0.20547387, 0.70755863, 0.08696745], dtype=float32), 'classes': 1}
{'probabilities': array([0.19193704, 0.7786191 , 0.02944385], dtype=float32), 'classes': 1}
{'probabilities': array([0.26921576, 0.6773188 , 0.053465

KeyboardInterrupt: 